# Install face-alignment package
Environment: Google colab

In [1]:
# Update pyorch to 0.4
# !pip install --upgrade torch

Requirement already up-to-date: torch in /home/bryan/.local/lib/python3.5/site-packages (0.4.0)


In [2]:
# !git init .
# !git remote add origin https://github.com/1adrianb/face-alignment.git
# !git fetch origin
# !git checkout master

Reinitialized existing Git repository in /home/bryan/faceswap-GAN/.git/
fatal: remote origin already exists.
^C
M	FaceSwap_GAN_v2.2_train_test.ipynb
M	networks/faceswap_gan_model.py
M	networks/losses.py
M	networks/nn_blocks.py
M	prep_binary_masks.ipynb
Already on 'master'
Your branch is behind 'origin/master' by 9 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)


In [3]:
# %%capture
# !apt update
# !apt install -y cmake

**Install dlib (CUDA enabled or CPU version)**

CUDA enabled dlib

In [4]:
# %%capture
# !git clone https://github.com/davisking/dlib.git dlib/
# !cd dlib/
# !python setup.py install –yes USE_AVX_INSTRUCTIONS –yes DLIB_USE_CUDA

dlib w/o CUDA

In [5]:
# CPU dlib
#!pip install dlib

In [6]:
!ls

binary_masks				  image_augmentation.py
converter				  instance_normalization.py
detector				  model_GAN_v2.py
dlib					  mtcnn_detect_face.py
dlib_video_face_detection.ipynb		  mtcnn_weights
faceA					  networks
faceB					  notes
FaceSwap_GAN_github.ipynb		  pixel_shuffler.py
FaceSwap_GAN_v2.1_train.ipynb		  prep_binary_masks.ipynb
FaceSwap_GAN_v2.2_train_test.ipynb	  __pycache__
FaceSwap_GAN_v2.2_video_conversion.ipynb  README.md
FaceSwap_GAN_v2_sz128_train.ipynb	  temp
FaceSwap_GAN_v2_test_img.ipynb		  training_data.py
FaceSwap_GAN_v2_test_video_MTCNN.ipynb	  umeyama.py
FaceSwap_GAN_v2_train.ipynb		  utils.py
FCN8s_keras.py


In [1]:
import torch
torch.__version__

'0.4.0'

# Generate binary masks

```bash
    Inputs:
        Images from ./faceA and ./faceB
    Outputs:
        Eyes binary masks, which are saved to ./binary_masks/faceA_eyes and ./binary_masks/faceB_eyes respectively
```

In [2]:
import face_alignment

In [3]:
import cv2
import numpy as np
from glob import glob
from pathlib import PurePath, Path
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
dir_faceA = "./faceA"
dir_faceB = "./faceB"
dir_bm_faceA_eyes = "./binary_masks/faceA_eyes"
dir_bm_faceB_eyes = "./binary_masks/faceB_eyes"

In [5]:
fns_faceA = glob("{dir_faceA}/*.*".format(dir_faceA=dir_faceA))
fns_faceB = glob("{dir_faceB}/*.*".format(dir_faceB=dir_faceB))

In [6]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, enable_cuda=True, flip_input=False)

In [7]:
# !mkdir -p binary_masks/faceA_eyes
Path("binary_masks/faceA_eyes").mkdir(parents=True, exist_ok=True)
# !mkdir -p binary_masks/faceB_eyes
Path("binary_masks/faceB_eyes").mkdir(parents=True, exist_ok=True)

In [9]:
fns_face_not_detected = []

for idx, fns in enumerate([fns_faceA, fns_faceB]):
    if idx == 0:
        save_path = dir_bm_faceA_eyes
    elif idx == 1:
        save_path = dir_bm_faceB_eyes     
    
    # create binary mask for each training image
    for fn in fns:
        raw_fn = PurePath(fn).parts[-1]

        x = plt.imread(fn)
        x = cv2.resize(x, (256,256))
        preds = fa.get_landmarks(x)
        
        if preds is not None:
            preds = preds[0]
            mask = np.zeros_like(x)
            
            # Draw right eye binary mask
            pnts_right = [(preds[i,0],preds[i,1]) for i in range(36,42)]
            hull = cv2.convexHull(np.array(pnts_right)).astype(np.int32)
            mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)

            # Draw left eye binary mask
            pnts_left = [(preds[i,0],preds[i,1]) for i in range(42,48)]
            hull = cv2.convexHull(np.array(pnts_left)).astype(np.int32)
            mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)
            
            mask = cv2.dilate(mask, np.ones((13,13), np.uint8), iterations=1)
            mask = cv2.GaussianBlur(mask, (7,7), 0)
            
        else:
            mask = np.zeros_like(x)
            print("No faces were detected in image '{fn}''".format(fn=fn))
            fns_face_not_detected.append(fn)
        
        plt.imsave(fname="{save_path}/{raw_fn}".format(save_path=save_path, raw_fn=raw_fn), arr=mask, format="jpg")

No faces were detected in image './faceA/632205874.jpg''
No faces were detected in image './faceA/885289262.jpg''
No faces were detected in image './faceA/507926366.jpg''
No faces were detected in image './faceA/542732898.jpg''
No faces were detected in image './faceA/600031554.jpg''
No faces were detected in image './faceA/667789008.jpg''
No faces were detected in image './faceA/495466518.jpg''
No faces were detected in image './faceA/872383186.jpg''
No faces were detected in image './faceA/610601728.jpg''
No faces were detected in image './faceA/578547478.jpg''
No faces were detected in image './faceA/652916944.jpg''
No faces were detected in image './faceA/614181114.jpg''
No faces were detected in image './faceA/887058936.jpg''
No faces were detected in image './faceA/656981288.jpg''
No faces were detected in image './faceA/610600380.jpg''
No faces were detected in image './faceA/578546622.jpg''
No faces were detected in image './faceA/837554168.jpg''
No faces were detected in image

In [10]:
num_faceA = len(glob(dir_faceA+"/*.*"))
num_faceB = len(glob(dir_faceB+"/*.*"))

print("Nuber of processed images: "+ str(num_faceA + num_faceB))
print("Number of image(s) with no face detected: " + str(len(fns_face_not_detected)))

Nuber of processed images: 672
Number of image(s) with no face detected: 67


# Randomly diaplay a face image and its result binary mask

In [11]:
face = np.random.choice(["A","B"])

dir_face = dir_faceA if face == "A" else dir_faceB
fns_face = fns_faceA if face == "A" else fns_faceB
num_face = len(glob(dir_face+"/*.*"))
rand_idx = np.random.randint(num_face)
rand_fn = fns_face[rand_idx]
raw_fn = PurePath(rand_fn).parts[-1]
mask_fn = "{dir_bm_faceA_eyes}/{raw_fn}".format(dir_bm_faceA_eyes=dir_bm_faceA_eyes, raw_fn=raw_fn) if face == "A" else "{dir_bm_faceB_eyes}/{raw_fn}".format(dir_bm_faceB_eyes=dir_bm_faceB_eyes, raw_fn=raw_fn)
im = plt.imread(rand_fn)
mask = plt.imread(mask_fn)

if rand_fn in fns_face_not_detected:
    print("========== No faces were detected in this image! ==========")

fig = plt.figure(figsize=(15,6))
plt.subplot(1,3,1)
plt.grid('off')
plt.imshow(im)
plt.subplot(1,3,2)
plt.grid('off')
plt.imshow(mask)
plt.subplot(1,3,3)
plt.grid('off')
plt.imshow((mask/255*im).astype(np.uint8))

#fa.get_landmarks(x)

SyntaxError: invalid syntax (<ipython-input-11-634501a82107>, line 9)

# Randomly diaplay an image that has no face detected

In [ ]:
num_no_face_img = len(fns_face_not_detected)
rand_idx = np.random.randint(num_no_face_img)
x = plt.imread(fns_face_not_detected[rand_idx])
#x = cv2.resize(x, (256,256))

plt.grid('off')
plt.imshow(x)

#fa.get_landmarks(x)